In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [3]:

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2023/06/18 19:59:06 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/18 19:59:06 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/06/18 19:59:06 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/cvd/projects/ml-ops/02-mlflow/mlruns/1', creation_time=1687107546138, experiment_id='1', last_update_time=1687107546138, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [2]:
# !wget -O  /home/cvd/projects/ml-ops/02-mlflow/data/green_tripdata_2021-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet
# !wget -O  /home/cvd/projects/ml-ops/02-mlflow/data/green_tripdata_2021-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet

In [4]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype(str)

    return df

In [5]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO']
# categorical = ['PULocationID', 'DOLocationID']
numeric = ['trip_distance']


dv = DictVectorizer()

train_dicts = df_train[categorical + numeric].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numeric].to_dict(orient='records')
X_val = dv.transform(val_dicts)


In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.75871520559622

In [14]:
with mlflow.start_run():

    mlflow.set_tag("developer", "john")
    mlflow.log_param("train-data-path",  "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-path",  "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param('alpha', alpha)

    lsr = Lasso(alpha)
    lsr.fit(X_train, y_train)

    y_pred = lsr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

In [12]:
rr = Ridge(alpha=0.001)
rr.fit(X_train, y_train)

y_pred = rr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.510909603240781

In [50]:
# with open('models/lin_reg.bin', 'wb') as f_out:
#     pickle.dump((dv, lr), f_out)